In [1]:
from durable.lang import *
import threading
import time
import datetime

with ruleset('test'):
    # antecedent
    @when_all(m.subject == 'World')
    def say_hello(c):
        # consequent
        print('test-> Hello {0}'.format(c.m.subject))

post('test', { 'subject': 'World' })

test-> Hello World


{'sid': '0', 'id': 'sid-0', '$s': 1}

In [2]:
with ruleset('animal'):
    @when_all(c.first << (m.predicate == 'eats') & (m.object == 'flies'),
              (m.predicate == 'lives') & (m.object == 'water') & (m.subject == c.first.subject))
    def frog(c):
        c.assert_fact({ 'subject': c.first.subject, 'predicate': 'is', 'object': 'frog' })

    @when_all(c.first << (m.predicate == 'eats') & (m.object == 'flies'),
              (m.predicate == 'lives') & (m.object == 'land') & (m.subject == c.first.subject))
    def chameleon(c):
        c.assert_fact({ 'subject': c.first.subject, 'predicate': 'is', 'object': 'chameleon' })

    @when_all((m.predicate == 'eats') & (m.object == 'worms'))
    def bird(c):
        c.assert_fact({ 'subject': c.m.subject, 'predicate': 'is', 'object': 'bird' })

    @when_all((m.predicate == 'is') & (m.object == 'frog'))
    def green(c):
        c.assert_fact({ 'subject': c.m.subject, 'predicate': 'is', 'object': 'green' })

    @when_all((m.predicate == 'is') & (m.object == 'chameleon'))
    def grey(c):
        c.assert_fact({ 'subject': c.m.subject, 'predicate': 'is', 'object': 'grey' })

    @when_all((m.predicate == 'is') & (m.object == 'bird'))
    def black(c):
        c.assert_fact({ 'subject': c.m.subject, 'predicate': 'is', 'object': 'black' })

    @when_all(+m.subject)
    def output(c):
        print ('animal-> Fact: {0} {1} {2}'.format(c.m.subject, c.m.predicate, c.m.object))

assert_fact('animal', { 'subject': 'Kermit', 'predicate': 'eats', 'object': 'flies' })
assert_fact('animal', { 'subject': 'Kermit', 'predicate': 'lives', 'object': 'water' })
assert_fact('animal', { 'subject': 'Greedy', 'predicate': 'eats', 'object': 'flies' })
assert_fact('animal', { 'subject': 'Greedy', 'predicate': 'lives', 'object': 'land' })
assert_fact('animal', { 'subject': 'Tweety', 'predicate': 'eats', 'object': 'worms' })


animal-> Fact: Kermit eats flies
animal-> Fact: Kermit is green
animal-> Fact: Kermit is frog
animal-> Fact: Kermit lives water
animal-> Fact: Greedy eats flies
animal-> Fact: Greedy is grey
animal-> Fact: Greedy is chameleon
animal-> Fact: Greedy lives land
animal-> Fact: Tweety is black
animal-> Fact: Tweety is bird
animal-> Fact: Tweety eats worms


{'sid': '0', 'id': 'sid-0', '$s': 1}

In [3]:
with ruleset('expense5'):
    @when_all(c.bill << (m.t == 'bill') & (m.invoice.amount > 50),
              c.account << (m.t == 'account') & (m.payment.invoice.amount == c.bill.invoice.amount))
    def approved(c):
        print ('expense5-> bill amount: {0}'.format(c.bill.invoice.amount))
        print ('expense5-> account payment amount: {0}'.format(c.account.payment.invoice.amount))
        
post('expense5', {'t': 'bill', 'invoice': {'amount': 100}})
post('expense5', {'t': 'account', 'payment': {'invoice': {'amount': 100}}})

expense5-> bill amount: 100
expense5-> account payment amount: 100


{'sid': '0', 'id': 'sid-0', '$s': 1}

In [4]:
with ruleset('bookstore'):
    # this rule will trigger for events with status
    @when_all(+m.status)
    def event(c):
        print('bookstore-> Reference {0} status {1}'.format(c.m.reference, c.m.status))

    @when_all(+m.name)
    def fact(c):
        print('bookstore-> Added {0}'.format(c.m.name))
        
    # this rule will be triggered when the fact is retracted
    @when_all(none(+m.name))
    def empty(c):
        print('bookstore-> No books')

assert_fact('bookstore', {
    'name': 'The new book',
    'seller': 'bookstore',
    'reference': '75323',
    'price': 500
})

bookstore-> Added The new book


{'sid': '0', 'id': 'sid-0', '$s': 1}

In [1]:
from durable.lang import *

with ruleset('strings'):
    @when_all(m.subject.matches('hello.*'))
    def starts_with(c):
        print ('string starts with hello -> {0}'.format(c.m.subject))

    @when_all(m.subject.matches('.*hello'))
    def ends_with(c):
        print ('string ends with hello -> {0}'.format(c.m.subject))

    @when_all(m.subject.imatches('.*hello.*'))
    def contains(c):
        print ('string contains hello (case insensitive) -> {0}'.format(c.m.subject))
    
assert_fact('strings', { 'subject': 'HELLO world' })
assert_fact('strings', { 'subject': 'world hello' })
assert_fact('strings', { 'subject': 'hello hi' })
assert_fact('strings', { 'subject': 'has Hello string' })
# assert_fact('strings', { 'subject': 'does not match' })


string contains hello (case insensitive) -> HELLO world
string ends with hello -> world hello
string contains hello (case insensitive) -> world hello
string starts with hello -> hello hi
string contains hello (case insensitive) -> hello hi
string contains hello (case insensitive) -> has Hello string


{'sid': '0', 'id': 'sid-0', '$s': 1}

In [16]:
from durable.lang import *
from durable.engine import Host

def create_host():
    import durable.lang as dlang
    
    host = Host()
    ruleset_definitions = {}
    for name, rset in dlang._rulesets.items():
        full_name, ruleset_definition = rset.define()
        ruleset_definitions[full_name] = ruleset_definition

    host.register_rulesets(ruleset_definitions)
    return host

with ruleset('wordnet'):
    @when_all(m.subject.matches('.*thing.*, entity.*'))
    def matches(c):
        c.s.matches=True
        print ('string matches pat -> {0}'.format(c.m.subject))
    
    @when_all(m.amod.anyItem(item.matches('.*thing.*, entity.*')))
    def amod_with(c):
        c.s.amod_with=True
        print ('amod pat -> {0}'.format(c.m.amod))
    
    @when_all(m.subject.imatches('.*thing.*'))
    def contains(c):        
        print ('string contains pat -> {0}'.format(c.m.subject))

host=create_host()
# assert_fact
r1=host.post('wordnet', { 'subject': 'main.n.01, body_of_water.n.01, thing.n.12, physical_entity.n.01, entity.n.01'})
r2=host.post('wordnet', { 'amod': ['main.n.01, body_of_water.n.01, thing.n.12, physical_entity.n.01, entity.n.01']})
r1, r2

string matches pat -> main.n.01, body_of_water.n.01, thing.n.12, physical_entity.n.01, entity.n.01
amod pat -> ['main.n.01, body_of_water.n.01, thing.n.12, physical_entity.n.01, entity.n.01']


({'sid': '0', 'id': 'sid-0', '$s': 1, 'matches': True},
 {'sid': '0', 'id': 'sid-0', '$s': 1, 'matches': True, 'amod_with': True})

In [8]:
import glob
import rasa.utils.io as io_utils
intents=[]
for f in glob.glob('/pi/stack/conf/ruleset_*.json'):
    rules = io_utils.read_json_file(f)
    for rule in rules:
        intents.append({rule['intent']: {'triggers': rule['action'], 'rules':rule['rules']}})
intents

[{'behave_purpose': {'triggers': 'action_log_commevent',
   'rules': ['behave_purpose']}}]

In [11]:
from sagas.nlu.corenlp_parser import get_chunks
from sagas.nlu.rules_meta import build_meta
from sagas.nlu.inspector_common import Inspector, Context, non_spaces
from sagas.conf.conf import cf
from sagas.nlu.utils import fix_sents
from sagas.nlu.uni_remote import dep_parse

def parse_sents(data):
    sents, source=data['sents'], data['lang']
    sents=fix_sents(sents, source)
    engine=cf.engine(source)
    doc_jsonify, resp = dep_parse(sents, source, engine, ['predicts'])
    return doc_jsonify, resp

lang='en'
data = {'lang': lang, "sents": 'I want to play music.', 'engine': cf.engine(lang)}
doc_jsonify,_=parse_sents(data)
rs = get_chunks(doc_jsonify)

if lang in non_spaces:
    delim=''
else:
    delim=' '
for serial, r in enumerate(rs):
    meta = build_meta(r, data)
    domains=r['domains']
    # print([(x[0], x[2]) for x in domains])
    #
    keys={x[0] for x in domains}
    grp=lambda p, idx: [x[idx] for x in domains if x[0]==p]    
    tokens={x:grp(x,2) for x in keys}
    words={x:delim.join(grp(x,2)) for x in keys}
    lemmas={x:delim.join(grp(x,3)) for x in keys}
    print('meta keys', meta.keys())
    print('tokens', tokens)
    print('words', meta['word'], words)
    print('lemmas', lemmas)
    # 
    ctx=Context(meta, domains)
    # print(ctx.lemmas)
    print('chunks', ctx._chunks)

.. request is {'lang': 'en', 'sents': 'I want to play music.', 'engine': 'corenlp', 'pipelines': ['predicts']}
meta keys dict_keys(['rel', 'lemma', 'word', 'stems', 'lang', 'sents', 'engine'])
tokens {'nsubj': ['I'], 'xcomp': ['play'], 'punct': ['.'], 'head_root': ['.']}
words want {'nsubj': 'I', 'xcomp': 'play', 'punct': '.', 'head_root': '.'}
lemmas {'nsubj': 'I', 'xcomp': 'play', 'punct': '.', 'head_root': '.'}
chunks [nsubj: I, xcomp: to play music, punct: ., head_root: .]
